In [ ]:
import torch as th
from torch import nn
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt

In [ ]:
class LinearRegression(nn.Module):
    def __init__(self, device="cpu"):
        self.weights = None
        self.device = device

    def fit(self, X, y):
        # Go ahead and rock it!
        X = X.to(self.device)
        X = X.to(self.device)
        
        X = th.cat([X, th.ones_like(y).to(self.device)], dim=1)

        rows, cols = X.size()
        self.weights = th.matmul(
            th.matmul(
                th.inverse(
                    th.matmul(
                        th.transpose(X, 0, 1),
                        X)),
                th.transpose(X, 0, 1)),
            y)

    def predict(self, X):
        # Write your inference here!
        X = th.cat([X, th.ones((X.size()[0], 1))], dim=1)
        return th.matmul(X, self.weights)

In [ ]:
X, y = make_regression(n_samples=100, n_features=1, n_informative=1, bias=1, noise=35)
y = y.reshape((-1, 1))

In [ ]:
plt.scatter(X, y)

In [ ]:
lin_reg = LinearRegression()
X, y = th.from_numpy(X), th.from_numpy(y)
lin_reg.fit(X, y)

In [ ]:
plt.scatter(X, y)
plt.plot(X.numpy(), lin_reg.predict(X), color='orange')

In [ ]:
from time import time

no_inputs = [10**i for i in range(2, 8)]
results_torch = []

#Let's benchmark it!
for no_samples in no_inputs:
    X, y = make_regression(n_samples=no_samples, n_features=10, n_informative=7, bias=1, noise=100)
    y = y.reshape((-1, 1))
    X, y = th.from_numpy(X), th.from_numpy(y)

    lin_reg = LinearRegression()
    start = time()    
    lin_reg.fit(X, y)
    end = time()
    results_torch.append(end - start)

In [ ]:
plt.plot(no_inputs, results_torch)

In [ ]:
import numpy as np

class NumpyLinearRegression:
    def __init__(self):
        self.weights = None
        
    
    def fit(self, X, y):
        # Go ahead and rock it!
        X = np.column_stack((X, np.ones_like(y)))
        rows, cols = X.shape
    
        self.weights = np.matmul(
            np.matmul(
                np.linalg.inv(
                    np.matmul(
                        np.transpose(X),
                        X)),
                np.transpose(X)),
            y)

    def predict(self, X):
        X = np.column_stack((X, np.ones_like(y)))
        return np.matmul(X, self.weights)


In [ ]:
X, y = make_regression(n_samples=100, n_features=1, n_informative=7, bias=1, noise=100)
np_lin_reg = NumpyLinearRegression()
np_lin_reg.fit(X, y)

In [ ]:
plt.scatter(X, y)
plt.plot(X, np_lin_reg.predict(X), color='orange')

In [ ]:
results_numpy = []

#Let's benchmark it!
for no_samples in no_inputs:
    X, y = make_regression(n_samples=no_samples, n_features=10, n_informative=7, bias=1, noise=100)
    reg = NumpyLinearRegression()
    start = time()    
    reg.fit(X, y)
    end = time()
    results_numpy.append(end - start)

In [ ]:
results_torch_cuda = []
for no_samples in no_inputs:
    X, y = make_regression(n_samples=no_samples, n_features=10, n_informative=7, bias=1, noise=100)
    y = y.reshape((-1, 1))
    reg = LinearRegression(device="cuda")
    X, y = th.from_numpy(X), th.from_numpy(y)
    X = X.cuda()
    y = y.cuda()
    start = time()
    reg.fit(X, y)
    end = time()
    results_torch_cuda.append(end - start)

In [ ]:
plt.plot(no_inputs, results_torch_cuda, label="torch-cuda")
plt.plot(no_inputs, results_torch, label="torch-cpu")
plt.plot(no_inputs, results_numpy, label="numpy")
plt.legend()